In [1]:
!pip install monsterapi==1.0.2b3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.5 MB/s eta 0:00:00ta 0:00:01


In [2]:
auth_token = "b6a97d3b-35d0-4720-a44c-59ee33dbc25b"

In [3]:
from monsterapi import client as mclient

/Users/kingsleyzhong/Documents/treehacks24/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
import json

url = "https://de6f507c-7cbd-40c0-903a-63987f3866fe.monsterapi.ai"

# assert status_ret.get("status") == "live", "Please wait until status is live!"

service_client = mclient(api_key=auth_token, base_url=url)

payload = {
    "input_variables": {"prompt": "What's up?"},
    "stream": False,
    "temperature": 0.7,
    "max_tokens": 2048,
}

output = service_client.generate(model="deploy-llm", data=payload)

if payload.get("stream"):
    for i in output:
        print(i[0])
else:
    print(json.loads(output)["text"][0])

/Users/kingsleyzhong/Documents/treehacks24/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'de6f507c-7cbd-40c0-903a-63987f3866fe.monsterapi.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


¡Hola! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with any questions you have about Spanish grammar or vocabulary. Is there something specific you'd like to know or practice? I'm here to help you improve your Spanish!

¡Que tengas un buen día!

(By the way, "What's up?" can be translated as "¿Qué pasa?" in Spanish.)
